In [50]:
import numpy as np

np.random.seed(2)

N = 77
ch = 256
H = 23
W = 41

h = np.random.randn(N, ch, H, W).astype(np.float32)
c = np.random.randint(low=0, high=10, size=(N,))

In [53]:
from resnet_layers import ConditionalBatchNorm2d

bn = ConditionalBatchNorm2d(ch, 10)
h_t = torch.from_numpy(h)
c_t = torch.from_numpy(c)

bn(h_t, c_t)
bn(h_t, c_t)

tensor([[[[-4.1446e-01, -5.4818e-02, -2.1298e+00,  ...,  1.3255e-03,
            5.4239e-01, -3.1145e-01],
          [ 7.7050e-01, -1.8624e+00,  1.7284e+00,  ..., -2.0273e+00,
           -1.8771e-01, -7.5720e-02],
          [ 8.2407e-01,  1.2466e+00, -4.0162e-01,  ..., -1.0177e+00,
            4.3269e-01, -3.2150e-01],
          ...,
          [-2.0105e-01,  6.7088e-01, -3.0793e-01,  ..., -1.6287e+00,
            5.8981e-01,  1.8062e+00],
          [-1.9326e-01, -1.3299e+00,  6.8262e-01,  ..., -5.4956e-03,
           -4.0785e-01, -2.1811e-01],
          [ 4.1764e-01, -1.4575e-01, -5.2767e-01,  ...,  2.3030e+00,
           -6.3172e-01,  6.3353e-01]],

         [[-9.6552e-01,  3.1932e+00, -1.3331e+00,  ..., -2.4700e-01,
            9.9385e-01,  7.9526e-01],
          [-5.1985e-01, -4.2625e-01, -5.0251e-02,  ..., -4.9019e-01,
           -9.3451e-02,  7.5595e-01],
          [ 6.5197e-01,  3.8512e-02,  7.4391e-01,  ...,  3.5073e-01,
            5.2334e-01, -1.5482e+00],
          ...,
     

In [14]:
sd = optim.state_dict()
print(sd)

optim_2 = torch.optim.Adam(model.parameters(), lr = .001)
print(optim_2.state_dict())

optim_2.load_state_dict(sd)
print(optim_2.state_dict())



{'state': {}, 'param_groups': [{'lr': 1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}
{'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}
{'state': {}, 'param_groups': [{'lr': 1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}


In [167]:
import torchvision
import torchvision.transforms as transforms


trainset = torchvision.datasets.CIFAR10(
    root=DEFAULT_SN_GAN_DATA_PATH, 
    train=True,
    download=True, 
    # transform=transforms.ToTensor()
)

trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=64,
    num_workers=1,
    shuffle=True
)

Files already downloaded and verified


In [213]:
n = 100
image = trainset[n][0]

arr = torchvision.transforms.ToTensor()(image)

arr = arr*2 - 1
print(arr.shape)

img = torchvision.transforms.ToPILImage()(arr)

img.show()

torch.Size([3, 32, 32])


tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],

In [34]:
class ConditionalBatchNorm2d(nn.Module):
    # Thanks to https://github.com/Kaixhin for this layer
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.bn = nn.BatchNorm2d(num_features, affine=False)
        self.embed = nn.Embedding(num_classes, num_features * 2)
        self.embed.weight.data[:, :num_features] = 1  
        self.embed.weight.data[:, num_features:] = 0  

    def forward(self, x, y):
        out = self.bn(x)
        gamma, beta = self.embed(y).chunk(2, 1)
        out = gamma.view(-1, self.num_features, 1, 1) * out + beta.view(-1, self.num_features, 1, 1)
        return out
    
ConditionalBatchNorm2d(128, 10).embed.weight.data

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])

In [3]:
from cifar10_models import Cifar10Discriminator
import torch 


dis = Cifar10Discriminator(n_classes=5)

im = torch.rand(94, 3, 32, 32) * 2 - 1
labels = torch.ones((94, ), dtype=torch.long)

dis(im, labels)

p shape:  torch.Size([94, 128])
label weight shape: torch.Size([94, 128])
label_weights * p shape: torch.Size([94, 128])
torch.Size([94, 1])


tensor([[-18.4368],
        [-17.7502],
        [-18.1937],
        [-18.2506],
        [-17.7299],
        [-17.4220],
        [-17.7976],
        [-17.4032],
        [-18.1591],
        [-17.5930],
        [-17.7073],
        [-17.3667],
        [-17.6906],
        [-18.0051],
        [-18.2365],
        [-17.4930],
        [-18.0018],
        [-17.9581],
        [-17.9253],
        [-17.1292],
        [-17.3356],
        [-17.7500],
        [-17.6907],
        [-18.5198],
        [-17.4648],
        [-17.8622],
        [-17.9219],
        [-18.0552],
        [-17.3677],
        [-18.0826],
        [-17.3437],
        [-17.3026],
        [-18.3996],
        [-17.8657],
        [-17.5924],
        [-17.0631],
        [-17.8615],
        [-17.8787],
        [-17.6510],
        [-17.2374],
        [-17.5856],
        [-18.1056],
        [-17.9741],
        [-17.7555],
        [-17.0572],
        [-17.3080],
        [-17.3251],
        [-17.7208],
        [-17.6329],
        [-17.8323],
